In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy import interpolate
import csv
import datetime
from scipy.optimize import curve_fit
from numpy import arange
from matplotlib import pyplot
import seaborn as sns

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
#parameter values - constants
m = 5/3
mod_start = 90
tstep = 5
krea = 0.00025
delt = 30
L = 4660
W = 316

In [ ]:
#metabolism model inputs
metab_inputs = pd.read_csv('twost_solute_model_inputs.csv')
metab_inputs['date_time'] =  pd.to_datetime(metab_inputs['date_time'], format='%d/%m/%Y %H:%M')
metab_inputs['us'] = 0.0047 * (metab_inputs['Q'] ** 0.8699)
metab_inputs['uadv'] = metab_inputs['us'] 
metab_inputs['cadv'] = m * metab_inputs['uadv']
metab_inputs['Tflowadv_emp'] = (L / metab_inputs['cadv']) / 60 #unit:min
metab_inputs['Tsadv'] = (L / metab_inputs['us']) / 60 #unit:min
metab_inputs['Tuadv'] = (L / metab_inputs['uadv']) / 60 #unit:min
metab_inputs['dep'] = (141 + (0.676 * np.exp(0.0027*metab_inputs['Q'])) - (0.676 * np.exp(0.0027*85))) / 100 
#because depth is regulated

In [ ]:
#optimise Fadv by minimising residuals in GPP-PAR link function
#dcdt function
def calc_nep(t, delt, tstep, mod_start, Ci, Cobs, krea, Cs, dep, Tsadv): 

    tf = mod_start + t
    deltx = int(delt/tstep)
            
    Tsadv = Tsadv[tf]   
    # alpha = Fadv_val * Tsadv
    alpha = Fadv * Tsadv
    t_lag = round(tf - int(alpha/tstep))    

    nep = (
        ((Cobs[tf+deltx] - Cobs[tf]) / delt) - 
        ((Ci[t_lag] - Cobs[tf]) * (Qi[t_lag] / (Q[tf] * Tsadv))) -
        (krea * (Cs[tf] - Cobs[tf]))
        ) * dep[tf] * 1000

    return nep

list_ss_res = []
Fadv_values = np.linspace(0,1,100)

for value in range(len(Fadv_values)):

    Fadv = Fadv_values[value]

    timesteps = len(metab_inputs) - int(delt/tstep)
    NEPmodel = np.full((timesteps), np.nan)

    for time_step in range(timesteps - mod_start):

        Qi = metab_inputs['Qi'].tolist()
        Q = metab_inputs['Q'].tolist()
        Ci = metab_inputs['Ci'].tolist()
        PAR = metab_inputs['PAR'].tolist()
        Cs = metab_inputs['Cs'].tolist()
        dep = metab_inputs['dep'].tolist()
        Cobs= metab_inputs['Cobs'].tolist()
        Tsadv= metab_inputs['Tsadv'].tolist()
        Tflowadv_emp = metab_inputs['Tflowadv_emp'].tolist() 
        Tuadv= metab_inputs['Tuadv'].tolist()
        
        NEPmodel[time_step + mod_start] = calc_nep(t=time_step, delt=delt, tstep=tstep, 
        mod_start=mod_start, Ci=Ci, Cobs=Cobs, krea=krea, Cs=Cs, dep=dep, Tsadv=Tsadv)
        

    deltx = int(delt/tstep)
    NEPmodel = np.append(NEPmodel, np.repeat(np.nan, deltx))
    NEPmodel
    metab_inputs['NEP'] = NEPmodel[:]

    metab_inputs['NEP_gday'] = metab_inputs['NEP'] / 1000 *60 * 24
    metab_inputs['light_metab'] = np.where(metab_inputs['PAR_umol']>1,metab_inputs['NEP_gday'],999) 
    metab_inputs['dark_metab'] = np.where(metab_inputs['PAR_umol']<1,metab_inputs['NEP_gday'],np.NaN)
    metab_inputs = metab_inputs.reset_index()
    metab_inputs = metab_inputs.set_index('date_time')
    df_avg = metab_inputs.resample('D').mean()
    indices = ['2019-08-04','2019-08-08']
    df_avg.loc[indices[0]]['dark_metab'] = 0
    df_avg.loc[indices[1]]['dark_metab'] = 0

    metab_filt = metab_inputs.loc['2019-08-04':'2019-08-08']
    metab_filt['er_daily'] = np.nan

    for date in df_avg.index.strftime('%Y-%m-%d'):
            date_ids = metab_filt.index[metab_filt.index.strftime('%Y-%m-%d') == date]
            metab_filt['er_daily'][date_ids] = df_avg['dark_metab'][date]
            

    metab_filt['gpp_daily'] = np.where(metab_filt['light_metab'] == 999.0, 
    0, metab_filt['light_metab']-metab_filt['er_daily'])
    metab_filt['gpp_daily_light'] = np.where(metab_filt['light_metab'] == 999.0, 
    np.NaN, metab_filt['light_metab']-metab_filt['er_daily'])

    fitting_df = metab_filt[metab_filt['gpp_daily_light'].notna()]
    
    #non linear curve fit
    par = fitting_df['PAR_umol'].values
    gpp_daily = fitting_df['gpp_daily_light'].values

    def objective(x, a, b):
        return (a * x) / (b + x)

    # choose the input and output variables
    x, y = par, gpp_daily

    popt, pcov = curve_fit(objective, x, y)
    a, b = popt
    print('y = %.5f * x / %.5f + x' % (a, b))

    residuals = y- objective(x, *popt)
    ss_res = np.sum(residuals**2)  # minimise ss_res to optimise Fadv
    
    list_ss_res.append(ss_res)

 

In [ ]:
#Find Fadv for lowest ss_res
index_min = min(range(len(list_ss_res)), key=list_ss_res.__getitem__)
value_min = min(list_ss_res)
Fadv = Fadv_values[index_min]
Fadv

In [ ]:
# run the model again using optimised Fadv 
# metabolism model inputs
metab_inputs = pd.read_csv('twost_solute_model_inputs.csv')
metab_inputs['date_time'] =  pd.to_datetime(metab_inputs['date_time'], format='%d/%m/%Y %H:%M')
metab_inputs['us'] = 0.0047 * (metab_inputs['Q'] ** 0.8699)
metab_inputs['uadv'] = metab_inputs['us'] 
metab_inputs['cadv'] = m * metab_inputs['uadv']
metab_inputs['Tflowadv_emp'] = (L / metab_inputs['cadv']) / 60 #unit:min
metab_inputs['Tsadv'] = (L / metab_inputs['us']) / 60 #unit:min
metab_inputs['Tuadv'] = (L / metab_inputs['uadv']) / 60 #unit:min
metab_inputs['dep'] = (141 + (0.676 * np.exp(0.0027*metab_inputs['Q'])) - (0.676 * np.exp(0.0027*85))) / 100

In [ ]:
timesteps = len(metab_inputs) - int(delt/tstep)
NEPmodel = np.full((timesteps), np.nan)

for time_step in range(timesteps - mod_start):

    Qi = metab_inputs['Qi'].tolist()
    Q = metab_inputs['Q'].tolist()
    Ci = metab_inputs['Ci'].tolist()
    PAR = metab_inputs['PAR'].tolist()
    Cs = metab_inputs['Cs'].tolist()
    dep = metab_inputs['dep'].tolist()
    Cobs= metab_inputs['Cobs'].tolist()
    Tsadv= metab_inputs['Tsadv'].tolist()
    Tflowadv_emp = metab_inputs['Tflowadv_emp'].tolist() 
    Tuadv= metab_inputs['Tuadv'].tolist()
    
    NEPmodel[time_step + mod_start] = calc_nep(t=time_step, delt=delt, tstep=tstep, mod_start=mod_start, 
    Ci=Ci, Cobs=Cobs, krea=krea, Cs=Cs, dep=dep, Tsadv=Tsadv)


deltx = int(delt/tstep)
NEPmodel = np.append(NEPmodel, np.repeat(np.nan, deltx))
NEPmodel
metab_inputs['NEP'] = NEPmodel[:]

#plot NEP
fig, ax = plt.subplots(1,figsize=(10,5))
ax2 = ax.twinx()

ax.scatter('date_time', 'NEP',data=metab_inputs,color='blue', marker="v", s=10)
ax2.plot('date_time','PAR',data=metab_inputs,color = 'orange')
ax.set_xlim([datetime.date(2019, 8, 5), datetime.date(2019, 8, 8)])
ax.axhline(y=0, color='black', linestyle='-')
ax.set_ylabel('NEP (mg $O_2$ $m^{-2}$ $min^{-1}$)')
ax2.set_ylabel('PAR ($\mu$mol quanta $m^{-2}$ $s^{-1}$)')

plt.savefig('nep_twostADV.png')


In [ ]:
metab_inputs['NEP_gday'] = metab_inputs['NEP'] / 1000 *60 * 24
metab_inputs['light_metab'] = np.where(metab_inputs['PAR_umol']>1,metab_inputs['NEP_gday'],999) 
metab_inputs['dark_metab'] = np.where(metab_inputs['PAR_umol']<1,metab_inputs['NEP_gday'],np.NaN)
metab_inputs = metab_inputs.reset_index()
metab_inputs = metab_inputs.set_index('date_time')
df_avg = metab_inputs.resample('D').mean()
indices = ['2019-08-04','2019-08-08']
df_avg.loc[indices[0]]['dark_metab'] = 0
df_avg.loc[indices[1]]['dark_metab'] = 0

metab_filt = metab_inputs.loc['2019-08-04':'2019-08-08']
metab_filt['er_daily'] = np.nan

for date in df_avg.index.strftime('%Y-%m-%d'):
        date_ids = metab_filt.index[metab_filt.index.strftime('%Y-%m-%d') == date]
        metab_filt['er_daily'][date_ids] = df_avg['dark_metab'][date]
        

metab_filt['gpp_daily'] = np.where(metab_filt['light_metab'] == 999.0, 
0, metab_filt['light_metab']-metab_filt['er_daily'])
metab_filt['gpp_daily_light'] = np.where(metab_filt['light_metab'] == 999.0, 
np.NaN, metab_filt['light_metab']-metab_filt['er_daily'])
metab_filt.to_csv('two_st_out_adv.csv')

fitting_df = metab_filt[metab_filt['gpp_daily_light'].notna()]

#non linear curve fit
par = fitting_df['PAR_umol'].values
gpp_daily = fitting_df['gpp_daily_light'].values

def objective(x, a, b):
    return (a * x) / (b + x)

# choose the input and output variables
x, y = par, gpp_daily

popt, pcov = curve_fit(objective, x, y)
a, b = popt
print('y = %.5f * x / %.5f + x' % (a, b))

residuals = y- objective(x, *popt)
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y-np.mean(y))**2)
r_squared = 1 - (ss_res / ss_tot)

# plot input vs output
pyplot.scatter(x, y,marker='x',alpha=0.5, s = 8,c='k')
x_line = arange(min(x), max(x), 1)
y_line = objective(x_line, a, b)
pyplot.plot(x_line, y_line, '-',linewidth=2, color='coral')
plt.xlabel('PAR ($\mu$mol quanta $m^{-2}$ $s^{-1}$)',size=12)
plt.ylabel('GPP (g $O_2$ $m^{-2}$ $d^{-1}$)',size=12)

string = r'$GPP_{{max}} = %.1f, k_{{PAR}} = %.0f$'% (a, b)
plt.text(760,-1.5,string,size=12)
plt.text(870,-4.5,'$R^2$ = %.2f' % r_squared,size=12)
plt.yticks(np.arange(-15,26,5))
plt.tick_params(axis='both', labelsize=12)
sns.despine(offset=5, trim=True)
plt.savefig('gpp_par_twostADV.png',bbox_inches='tight',dpi=500)